In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

Using TensorFlow backend.


In [2]:
data = pd.read_csv('new_clean_tweet_model3.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]  

In [3]:
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: str(x))
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

120000
120000


In [4]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 44, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 44, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(80400, 44) (80400, 2)
(39600, 44) (39600, 2)


In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 19, batch_size=batch_size, verbose = 2)

Epoch 1/19
 - 321s - loss: 0.4901 - acc: 0.7617
Epoch 2/19
 - 324s - loss: 0.4382 - acc: 0.7948
Epoch 3/19
 - 334s - loss: 0.4216 - acc: 0.8038
Epoch 4/19
 - 572s - loss: 0.4086 - acc: 0.8096
Epoch 5/19
 - 458s - loss: 0.3973 - acc: 0.8160
Epoch 6/19
 - 447s - loss: 0.3875 - acc: 0.8210
Epoch 7/19
 - 445s - loss: 0.3788 - acc: 0.8260
Epoch 8/19
 - 315s - loss: 0.3673 - acc: 0.8319
Epoch 9/19
 - 304s - loss: 0.3584 - acc: 0.8379
Epoch 10/19
 - 497s - loss: 0.3483 - acc: 0.8416
Epoch 11/19
 - 320s - loss: 0.3389 - acc: 0.8473
Epoch 12/19
 - 343s - loss: 0.3297 - acc: 0.8529
Epoch 13/19
 - 318s - loss: 0.3210 - acc: 0.8563
Epoch 14/19
 - 326s - loss: 0.3130 - acc: 0.8598
Epoch 15/19
 - 373s - loss: 0.3049 - acc: 0.8652
Epoch 16/19
 - 324s - loss: 0.2970 - acc: 0.8681
Epoch 17/19
 - 325s - loss: 0.2892 - acc: 0.8709
Epoch 18/19
 - 380s - loss: 0.2830 - acc: 0.8754
Epoch 19/19
